# 1. Protein

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [ ]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/proteinmodel30');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans BCM
BCM: SMSE 6.70721003, MSLL -0.7674, NLPD 2.4637
Processing protein kmeans PoE
PoE: SMSE 26.51998942, MSLL 9.3805, NLPD 12.6116


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans RBCM
RBCM: SMSE 6.20729156, MSLL -0.8720, NLPD 2.3627


### 1.1.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 6.05652218, MSLL -0.4614, NLPD 2.7733 - q   0.20
TERBCM: SMSE 5.85902334, MSLL -0.7736, NLPD 2.4611 - q   0.40
TERBCM: SMSE 5.83755318, MSLL -0.8629, NLPD 2.3718 - q   0.60
TERBCM: SMSE 5.95576436, MSLL -0.8829, NLPD 2.3518 - q   0.80
TERBCM: SMSE 6.20729156, MSLL -0.8720, NLPD 2.3627 - q   1.00
TERBCM: SMSE 6.59058925, MSLL -0.8449, NLPD 2.3898 - q   1.20
TERBCM: SMSE 7.10295684, MSLL -0.8090, NLPD 2.4257 - q   1.40
TERBCM: SMSE 7.73861058, MSLL -0.7682, NLPD 2.4665 - q   1.60
TERBCM: SMSE 8.48825205, MSLL -0.7252, NLPD 2.5095 - q   1.80
TERBCM: SMSE 9.33938940, MSLL -0.6814, NLPD 2.5533 - q   2.00


### 1.1.3 Find single q

In [7]:
criterion = 'TERBCM';

In [8]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.8);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.8000


In [9]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [10]:
criterion = 'TERBCM';

In [12]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=1e4, lr=1e-3, lambda=1e3, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
qs;
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.59347
   1.18573
   0.83674
   0.86710
   1.06758
   0.86202
   0.83603
   0.84873
   1.24119
   0.83476
   0.83545
   1.05948
   1.02676
   1.01742
   0.98275
   0.95305
   0.82681
   0.92116
   0.88719
   0.83332
   0.96864
   0.84036
   1.12098
   0.85102
   0.91291
   1.17165
   0.86299
   0.98870
   1.24237
   0.85877

TERBCM: SMSE 6.08457943, MSLL -0.8991, NLPD 2.3356


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [13]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GRBCM
GRBCM: SMSE 6.54896199, MSLL -0.8536, NLPD 2.3811


### 1.2.2 Grid q

In [14]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGRBCM
TEGRBCM: SMSE 5.34005633, MSLL -1.0150, NLPD 2.2197 - q   0.20
TEGRBCM: SMSE 5.43758929, MSLL -0.9907, NLPD 2.2440 - q   0.40
TEGRBCM: SMSE 5.70843339, MSLL -0.9452, NLPD 2.2895 - q   0.60
TEGRBCM: SMSE 6.08545871, MSLL -0.8984, NLPD 2.3363 - q   0.80
TEGRBCM: SMSE 6.54896199, MSLL -0.8536, NLPD 2.3811 - q   1.00
TEGRBCM: SMSE 7.08744330, MSLL -0.8114, NLPD 2.4233 - q   1.20
TEGRBCM: SMSE 7.69044448, MSLL -0.7719, NLPD 2.4628 - q   1.40
TEGRBCM: SMSE 8.34684358, MSLL -0.7352, NLPD 2.4995 - q   1.60
TEGRBCM: SMSE 9.04469748, MSLL -0.7013, NLPD 2.5334 - q   1.80
TEGRBCM: SMSE 9.77163297, MSLL -0.6699, NLPD 2.5648 - q   2.00


### 1.2.3 Find single q

In [25]:
criterion = 'TEGRBCM';

In [16]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.2);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.2000


In [26]:
q

q =  1.0148


In [30]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [27]:
criterion = 'TEGRBCM';

In [28]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=1e4, lr=1e-3, lambda=1e3, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...


In [29]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.0205
   1.0195
   1.0233
   1.0180
   1.0279
   1.0234
   1.0160
   1.0219
   1.0384
   1.0229
   1.0238
   1.0341
   1.0236
   1.0247
   1.0274
   1.0229
   1.0202
   1.0253
   1.0263
   1.0218
   1.0291
   1.0166
   1.0258
   1.0181
   1.0197
   1.0299
   1.0175
   1.0303
   1.0391
   1.0193

TEGRBCM: SMSE 6.35399328, MSLL -0.8566, NLPD 2.3776


## About lambda

In [5]:
criterion = 'TEGRBCM';

In [ ]:
printf('Optimizing q\r\n');
for ldk=-5:10
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
ld =    1.0000e-05
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18957461, MSLL -1.0184, NLPD 2.2127
ld =    1.0000e-04
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18870262, MSLL -1.0183, NLPD 2.2128
ld =  0.0010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18798795, MSLL -1.0181, NLPD 2.2130
ld =  0.010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18910662, MSLL -1.0181, NLPD 2.2130
ld =  0.10000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18949789, MSLL -1.0177, NLPD 2.2134
ld =  1
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.18897132, MSLL -1.0180, NLPD 2.2131
ld =  10
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.19000257, MSLL -1.0185, NLPD 2.2126
ld =  100
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.19835618, MSLL -1.0209, NLPD 2.2102
ld =  1000


In [7]:
printf('Optimizing q\r\n');
for ldk=3:10
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
ld =  1000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.08577750, MSLL -1.0266, NLPD 2.2078
ld =  10000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 5.47430486, MSLL -0.9639, NLPD 2.2705
ld =  100000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 6.85273930, MSLL -0.8280, NLPD 2.4065
ld =  1000000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 10.00383418, MSLL -0.6597, NLPD 2.5747
ld =  10000000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 14.59004134, MSLL -0.5115, NLPD 2.7229
ld =  100000000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 18.82287530, MSLL -0.4139, NLPD 2.8206
ld =    1.0000e+09
Start preprocessing...





## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [13]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing protein kmeans GPoE
GPoE: SMSE 5.77323823, MSLL -0.8301, NLPD 2.4041


### 1.3.2 Grid q

In [15]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing protein kmeans TEGPoE
TEGPoE: SMSE 5.97491548, MSLL -0.1135, NLPD 3.1207 - q   0.20
TEGPoE: SMSE 5.92421169, MSLL -0.5878, NLPD 2.6464 - q   0.40
TEGPoE: SMSE 5.87360964, MSLL -0.7511, NLPD 2.4831 - q   0.60
TEGPoE: SMSE 5.82324054, MSLL -0.8141, NLPD 2.4201 - q   0.80
TEGPoE: SMSE 5.77323823, MSLL -0.8301, NLPD 2.4041 - q   1.00
TEGPoE: SMSE 5.72373746, MSLL -0.8199, NLPD 2.4143 - q   1.20
TEGPoE: SMSE 5.67487435, MSLL -0.7938, NLPD 2.4403 - q   1.40
TEGPoE: SMSE 5.62677937, MSLL -0.7575, NLPD 2.4767 - q   1.60
TEGPoE: SMSE 5.57957783, MSLL -0.7144, NLPD 2.5198 - q   1.80
TEGPoE: SMSE 5.53338811, MSLL -0.6667, NLPD 2.5675 - q   2.00


### 1.3.3 Find single q

In [16]:
criterion = 'TEGPoE';

In [17]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.0001);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.0001


In [18]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [ ]:
printf('Optimizing q\r\n');
iqs = q + 0.01* rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);

In [23]:
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

In [24]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.0306
   1.0204
   1.0152
   1.0169
   1.0237
   1.0204
   1.0238
   1.0173
   1.0228
   1.0172
   1.0240
   1.0240
   1.0165
   1.0246
   1.0226
   1.0236
   1.0169
   1.0199
   1.0245
   1.0171
   1.0166
   1.0237
   1.0260
   1.0206
   1.0171
   1.0249
   1.0164
   1.0168
   1.0269
   1.0187

TEGPoE: SMSE 5.76570150, MSLL -0.8308, NLPD 2.4034
